<a href="https://colab.research.google.com/github/d0rcsa/uplift-report/blob/internal-editable/uplift_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remerge uplift report

This notebook allows you to validate remerge provided uplift reporting numbers. To do so it downloads and analyses exported campaign and event data from S3. The campaign data contains all users that remerge marked to be part of an uplift test, the A/B group assignment, the timestamp of marking, conversion events (click, app open or similar) and their cost. The event data reflects the app event stream and includes events, their timestamp and revenue (if any). We calculate the incremental revenue and the iROAS in line with the [remerge whitepaper](https://drive.google.com/file/d/1PTJ93Cpjw1BeiVns8dTcs2zDDWmmjpdc/view). 

**Hint**: This notebook can be run in any Jupyter instance with enough space/memory, as a [Google Colab notebook](#Google-Colab-version) or as a standalone Python script. If you are using a copy of this notebook running on Colab or locally you can find the original template on [GitHub: remerge/uplift-report](https://github.com/remerge/uplift-report/blob/master/uplift_report_per_campaign.ipynb)

### Notebook configuration

For this notebook to work properly several variables in the [Configuration](#Configuration) section need to be be set: `customer`, `audience`, `
revenue_event`, `dates` and the AWS credentials. All of these will be provided by your remerge account manager. 

## Import packages

This notebook/script needs our Uplift Report helper library, as long as several other dependencies it brings with it


## Load helpers

In [3]:
!pip install 'xxhash==1.3.0'
!pip install 'pandas==0.24.2'
!pip install 'scipy==1.3.0'
!pip install 's3fs==0.3.0'
!pip install 'pyarrow==0.14.0'
!pip install 'partd==1.0.0'

     |████████████████████████████████| 51kB 3.4MB/s 
     |████████████████████████████████| 25.2MB 1.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled scipy-1.3.1
     |████████████████████████████████| 51kB 3.3MB/s 
  Created wheel for s3fs: filename=s3fs-0.3.0-py2.py3-none-any.whl size=17751 sha256=b805371abd824ff09c247cdd3ff83bb4e89e10cc54330e74a056dd8365315564
  Stored in directory: /root/.cache/pip/wheels/d4/5b/bb/5ab802653ead8635569805e742fffabccc01ec29517d445ae2
Successfully built s3fs
  Found existing installation: s3fs 0.3.3
    Uninstalling s3fs-0.3.3:
      Successfully uninstalled s3fs-0.3.3
     |████████████████████████████████| 31.6MB 9.1MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4039 sha256=fa382a96fde3f14b68c11f7be763f2e80757829fa9a6ef35767e3e0a1d063db5
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built locket


## Helpers

In [0]:
# Constants
__version__ = '5.4.15-internal'

# constants for groups
TEST = True
CONTROL = False

# Helpers
from datetime import datetime
import pandas as pd
import xxhash
import os
import scipy
import scipy.stats
import s3fs

from IPython.display import display



## Configuration

Set the customer name, audience and access credentials for the S3 bucket and path. Furthermore the event for which we want to evaluate the uplift needs to be set `revenue_event`.

In [0]:
# configure path and revenue event 
customer = ''
audiences = ['']
revenue_event = 'purchase'

# date range for the report
dates = pd.date_range(start='2019-01-01',end='2019-01-01')

# AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = ''
os.environ["AWS_SECRET_ACCESS_KEY"] = ''

# Configure the reporting output: 

# named groups that aggregate several campaigns
groups = {}

# show uplift results per campaign:
per_campaign_results = False

# base statistical calculations on unique converters instead of conversions
use_converters_for_significance = False

# enable deduplication heuristic for appsflyer
use_deduplication = False

In [0]:
cache_folder = "cache-v{0}".format(__version__)

# columns to load from CSV
bid_columns = ['ts', 'user_id', 'ab_test_group', 'campaign_id','cost_eur','event_type']
attribution_columns = ['ts', 'user_id', 'partner_event', 'revenue_eur']  


def load_marks_and_spend_data(customer, audiences, dates):
    df = pd.concat([read_csv(customer, audience, 'marks_and_spend', date, columns=bid_columns) for audience in audiences for date in dates],
                    ignore_index=True, verify_integrity=True)
    return df

def load_attribution_data(customer, audiences, dates, revenue_event, marks_and_spend_df, use_deduplication):
    marked_user_ids = marked(marks_and_spend_df)['user_id']
    df = pd.concat(
    [filter_by_user_ids(read_csv(customer, audience, 'attributions', date, attribution_columns, revenue_event, extract_revenue_events), marked_user_ids) for audience in audiences for date in dates],
    ignore_index=True, verify_integrity=True)

    # AppsFlyer sends some events twice - we want to remove the duplicates before the analysis
    if use_deduplication:
        df = drop_duplicates_in_attributions(df, pd.Timedelta('1 minute'))
    
    return df


def extract_revenue_events(df, revenue_event):
    """
    Only keep rows where the event is a revenue event and drop the partner_event column afterwards
    """
    df = df[df.partner_event == revenue_event]
    return df.drop(columns=['partner_event'])


def filter_by_user_ids(df, user_ids):
    if 'user_id' in df.columns:
        return df[df['user_id'].isin(user_ids)]
    else:
        return df


def improve_types(df):
    """
    Use more memory efficient types for ts,user_id and ab_test_group
    """
    df['ts'] = pd.to_datetime(df['ts'])
    df['ts'] = (df['ts'].astype('int64') / 1e9).astype('int32')
    df['user_id'] = df['user_id'].apply(xxhash.xxh64_intdigest).astype('int64')
    if 'ab_test_group' in df.columns:
        df['ab_test_group'] = df['ab_test_group'].transform(lambda g: g == 'test')
    return df


def path(customer, audience):
    return "s3://remerge-customers/{0}/uplift_data/{1}".format(customer, audience)


def to_parquet(df, filename):
    """
    parquet save and load helper
    """
    df.to_parquet(filename, engine='pyarrow')


def from_parquet_corrected(filename, s3_filename, fs, columns):
    """
    A little "hack" to convert old file on the fly
    """
    df = from_parquet(filename)
    update_cache = False
    if columns:
        to_drop = list(set(df.columns.values) - set(columns))
        if to_drop:
            df = df.drop(columns=to_drop)
            update_cache = True

    # remove events without a user id
    if df['user_id'].dtype == 'object':
        if df[df['user_id'].isnull()].empty == False or df[df['user_id'].str.len() != 36].empty == False:
            df = df[df['user_id'].str.len() == 36]
            update_cache = True

    if df['user_id'].dtype != 'int64':
        df = improve_types(df)
        update_cache = True

    if update_cache:
        print(datetime.now(), 'rewritting cached file with correct types (local and S3)', filename, s3_filename)
        to_parquet(df, filename)
        fs.put(filename, s3_filename)

    return df


def from_parquet(filename):
    return pd.read_parquet(filename, engine='pyarrow')


def read_csv(customer, audience, source, date, columns=None, revenue_event=None, chunk_filter_fn=None,
             chunk_size=10 ** 6):
    """
    Helper to download CSV files, convert to DF and print time needed.
    Caches files locally and on S3 to be reused.
    """
    now = datetime.now()

    date_str = date.strftime('%Y%m%d')

    filename = '{0}/{1}/{2}.csv.gz'.format(path(customer, audience), source, date_str)

    # local cache
    cache_dir = '{0}/{1}/{2}'.format(cache_folder, audience, source)
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    cache_filename = '{0}/{1}.parquet'.format(cache_dir, date_str)

    # s3 cache (useful if we don't have enough space on the Colab instance)
    s3_cache_filename = '{0}/{1}/{2}/{3}.parquet'.format(path(customer, audience), source, cache_folder, date_str)

    if source == 'attributions':
        cache_filename = '{0}/{1}-{2}.parquet'.format(cache_dir, date_str, revenue_event)

        # s3 cache (useful if we don't have enough space on the Colab instance)
        s3_cache_filename = '{0}/{1}/{2}/{3}-{4}.parquet' \
            .format(path(customer, audience), source, cache_folder, date_str, revenue_event)

    fs = s3fs.S3FileSystem(anon=False)
    fs.connect_timeout = 10  # defaults to 5
    fs.read_timeout = 30  # defaults to 15 

    if os.path.exists(cache_filename):
        print(now, 'loading from', cache_filename)
        return from_parquet_corrected(cache_filename, s3_cache_filename, fs, columns)

    if fs.exists(path=s3_cache_filename):
        print(now, 'loading from S3 cache', s3_cache_filename)

        # Download the file to local cache first to avoid timeouts during the load.
        # This way, if they happen, restart will be using local copies first.
        fs.get(s3_cache_filename, cache_filename)

        print(now, 'stored S3 cache file to local drive, loading', cache_filename)

        return from_parquet_corrected(cache_filename, s3_cache_filename, fs, columns)

    print(now, 'start loading CSV for', audience, source, date)

    read_csv_kwargs = {'chunksize': chunk_size}
    if columns:
        read_csv_kwargs['usecols'] = columns

    df = pd.DataFrame()

    if not fs.exists(path=filename):
        print(now, 'WARNING: no CSV file at for: ', audience, source, date, ', skipping the file: ', filename)
        return df

    for chunk in pd.read_csv(filename, escapechar='\\', low_memory=False, **read_csv_kwargs):
        if chunk_filter_fn:
            filtered_chunk = chunk_filter_fn(chunk, revenue_event)
        else:
            filtered_chunk = chunk

        # remove events without a user id
        filtered_chunk = filtered_chunk[filtered_chunk['user_id'].str.len() == 36]

        filtered_chunk = improve_types(filtered_chunk)

        df = pd.concat([df, filtered_chunk],
                       ignore_index=True, verify_integrity=True)

    print(datetime.now(), 'finished loading CSV for', date.strftime('%d.%m.%Y'),
          'took', datetime.now() - now)

    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    print(datetime.now(), 'caching local as parquet', cache_filename)
    to_parquet(df, cache_filename)

    # write it to the S3 cache folder as well
    print(datetime.now(), 'caching on S3 as parquet', s3_cache_filename)
    to_parquet(df, s3_cache_filename)

    return df


def calculate_ad_spend(df):
    ad_spend_micros = df[(df.event_type == 'buying_conversion') & (df.ab_test_group == TEST)]['cost_eur'].sum()
    return ad_spend_micros / 10 ** 6


def marked(df):
    """
    The dataframe created by `marked` will contain all mark events. Remerge marks users per campaign. If a user was
    marked once for an audience he will have the same group allocation for consecutive marks (different campaigns)
    unless manually reset on audience level.
    """
    if df.empty:
        return df
    
    mark_df = df[df.event_type == 'mark']

    # we dont need the event_type anymore (to save memory)
    mark_df = mark_df.drop(columns=['event_type'])

    sorted_mark_df = mark_df.sort_values('ts')

    depuplicated_mark_df = sorted_mark_df.drop_duplicates(['user_id'])

    return depuplicated_mark_df


def merge(mark_df, attributions_df):
    """
    `merge` joins the marked users with the revenue events and excludes any revenue event that happened before the
    user was marked.
    """
    merged_df = pd.merge(attributions_df, mark_df, on='user_id')

    return merged_df[merged_df.ts_x > merged_df.ts_y]


def drop_duplicates_in_attributions(df, max_timedelta):
    """
    # Clean the data
    Due to some inconsistencies in the measurement we need to clean the data before analysis.
    ### Remove duplicated events coming from AppsFlyer
    AppsFlyer is sending us two revenue events if they attribute the event to us. One of the events they send us does
    not contain attribution information and the other one does. Sadly, it is not possible for us to distinguish
    correctly if an event is a duplicate or if the user actually triggered two events with nearly the same information.
    Therefore we rely on a heuristic. We consider an event a duplicate if the user and revenue are equal and the events
    are less than a minute apart.
    """
    sorted = df.sort_values(['user_id', 'revenue_eur'])

    # Get values of the previous row
    sorted['last_ts'] = sorted['ts'].shift(1)
    sorted['last_user_id'] = sorted['user_id'].shift(1)
    sorted['last_revenue'] = sorted['revenue_eur'].shift(1)

    # Remove rows if the previous row has the same revenue_eur and user id and the ts are less than max_timedelta apart
    filtered = sorted[
        (sorted['user_id'] != sorted['last_user_id']) |
        (sorted['revenue_eur'] != sorted['last_revenue']) |
        ((pd.to_datetime(sorted['ts']) - pd.to_datetime(sorted['last_ts'])) > max_timedelta)]

    return filtered[['ts', 'user_id', 'revenue_eur']]


def uplift(marks_and_spend_df, attributions_df, index_name, use_converters_for_significance, m_hypothesis=1):
    """
    # Uplift Calculation
    We calculate the incremental revenue and the iROAS in line with the
    [remerge whitepaper](https://drive.google.com/file/d/1PTJ93Cpjw1BeiVns8dTcs2zDDWmmjpdc/view). Afterwards we run a
    [chi squared test](https://en.wikipedia.org/wiki/Chi-squared_test) on the results to test for significance of the
    results, comparing conversion to per group uniques.
    """
    # filter for mark events
    marks_df = marked(marks_and_spend_df)

    # calculate group sizes
    test_group_size = marks_df[marks_df['ab_test_group'] == TEST]['user_id'].nunique()
    if test_group_size == 0:
        print("WARNING: No users marked as test for ", index_name, 'skipping.. ')
        return None

    control_group_size = marks_df[marks_df['ab_test_group'] == CONTROL]['user_id'].nunique()
    if control_group_size == 0:
        print("WARNING: No users marked as control for ", index_name, 'skipping.. ')
        return None

    # join marks and revenue events    
    merged_df = merge(marks_df, attributions_df)
    grouped_revenue = merged_df.groupby(by='ab_test_group')

    # init all KPIs with 0s first:
    test_revenue_micros = 0
    test_conversions = 0
    test_converters = 0

    control_revenue_micros = 0
    control_conversions = 0
    control_converters = 0

    # we might not have any events for a certain group in the time-period,
    if TEST in grouped_revenue.groups:
        test_revenue_df = grouped_revenue.get_group(TEST)
        test_revenue_micros = test_revenue_df['revenue_eur'].sum()
        # test_conversions = test_revenue_df['partner_event'].count()
        # as we filtered by revenue event and dropped the column we can just use
        test_conversions = test_revenue_df['user_id'].count()
        test_converters = test_revenue_df['user_id'].nunique()

    if CONTROL in grouped_revenue.groups:
        control_revenue_df = grouped_revenue.get_group(CONTROL)
        control_revenue_micros = control_revenue_df['revenue_eur'].sum()
        # control_conversions = control_revenue_df['partner_event'].count()
        # as we filtered by revenue event and dropped the column we can just use
        control_conversions = control_revenue_df['user_id'].count()
        control_converters = control_revenue_df['user_id'].nunique()

    # calculate KPIs
    test_revenue = test_revenue_micros / 10 ** 6
    control_revenue = control_revenue_micros / 10 ** 6

    ratio = float(test_group_size) / float(control_group_size)
    scaled_control_conversions = float(control_conversions) * ratio
    scaled_control_revenue_micros = float(control_revenue_micros) * ratio
    incremental_conversions = test_conversions - scaled_control_conversions
    incremental_revenue_micros = test_revenue_micros - scaled_control_revenue_micros
    incremental_revenue = incremental_revenue_micros / 10 ** 6
    incremental_converters = test_converters - control_converters * ratio

    # calculate the ad spend        
    ad_spend = calculate_ad_spend(marks_and_spend_df)

    iroas = incremental_revenue / ad_spend
    icpa = ad_spend / incremental_conversions
    cost_per_incremental_converter = ad_spend / incremental_converters

    rev_per_conversion_test = 0
    rev_per_conversion_control = 0
    if test_conversions > 0:
        rev_per_conversion_test = test_revenue / test_conversions
    if control_conversions > 0:
        rev_per_conversion_control = control_revenue / control_conversions

    test_cvr = test_conversions / test_group_size
    control_cvr = control_conversions / control_group_size

    uplift = 0
    if control_cvr > 0:
        uplift = test_cvr / control_cvr - 1

    # calculate statistical significance
    control_successes, test_successes = control_conversions, test_conversions
    if use_converters_for_significance or max(test_cvr, control_cvr) > 1.0:
        control_successes, test_successes = control_converters, test_converters
    chi_df = pd.DataFrame({
        "conversions": [control_successes, test_successes],
        "total": [control_group_size, test_group_size]
    }, index=['control', 'test'])
    # CHI square calculation will fail with insufficient data
    # Fallback to no significance
    try:
        chi, p, _, _ = scipy.stats.chi2_contingency(
            pd.concat([chi_df.total - chi_df.conversions, chi_df.conversions], axis=1), correction=False)
    except:
        chi, p = 0, 1.0

    # bonferroni correction with equal weights - if we have multiple hypothesis:
    # https://en.wikipedia.org/wiki/Bonferroni_correction
    significant = p < 0.05 / m_hypothesis

    dataframe_dict = {
        "ad spend": ad_spend,
        "total revenue": test_revenue + control_revenue,
        "test group size": test_group_size,
        "test conversions": test_conversions,
        "test converters": test_converters,
        "test revenue": test_revenue,
        "control group size": control_group_size,
        "control conversions": control_conversions,
        "control_converters": control_converters,
        "control revenue": control_revenue,
        "ratio test/control": ratio,
        "control conversions (scaled)": scaled_control_conversions,
        "control revenue (scaled)": scaled_control_revenue_micros / 10 ** 6,
        "incremental conversions": incremental_conversions,
        "incremental converters": incremental_converters,
        "incremental revenue": incremental_revenue,
        "rev/conversions test": rev_per_conversion_test,
        "rev/conversions control": rev_per_conversion_control,
        "test CVR": test_cvr,
        "control CVR": control_cvr,
        "CVR Uplift": uplift,
        "iROAS": iroas,
        "cost per incr. converter": cost_per_incremental_converter,
        "iCPA": icpa,
        "chi^2": chi,
        "p-value": p,
        "significant": significant
    }

    # show results as a dataframe
    return pd.DataFrame(
        dataframe_dict,
        index=[index_name],
    ).transpose()


def uplift_report(marks_and_spend_df, attributions_df, groups, per_campaign_results, use_converters_for_significance):
    """
    Calculate and display uplift report for the data set as a whole
    This takes the whole data set and calculates uplift KPIs.
    """
    # calculate the total result:
    report_df = uplift(marks_and_spend_df, attributions_df, "total", use_converters_for_significance)

    # if there are groups filter the events against the per campaign groups and generate report
    if report_df is not None and len(groups) > 0:
        for name, campaigns in groups.items():
            group_df = marks_and_spend_df[marks_and_spend_df.campaign_id.isin(campaigns)]
            report_df[name] = uplift(group_df, attributions_df, name, use_converters_for_significance, len(groups))

    if report_df is not None and per_campaign_results:
        campaigns = marks_and_spend_df['campaign_id'].unique()
        for campaign in campaigns:
            name = "c_{0}".format(campaign)
            campaign_df = marks_and_spend_df[marks_and_spend_df.campaign_id == campaign]
            report_df[name] = uplift(campaign_df, attributions_df, name, use_converters_for_significance,
                                     len(campaigns))
    return report_df


def export_csv(df, file_name):
    df.to_csv(file_name)

    print('Stored results as a local CSV file: %s' % file_name)

    try:
        import google.colab

        print('The download of the results file should start automatically')
        google.colab.files.download(file_name)
    except ImportError:
        # We are not in the collab, no need to run the download
        pass

def overview_row(customer, audiences, dates, total):
    return list([
        customer,
        ",".join(audiences),
        dates[0].strftime('%Y-%m-%d'),
        dates[-1].strftime('%Y-%m-%d'),
        __version__,
    ]) + list(total.values)

def export_to_overview(customer, audiences, dates, report):
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1je3b1g6Yg2B-bmwx6CP5akcrsj9DzQcs9PnKHoKoioQ/edit').sheet1
    row = overview_row(customer, audiences, dates, report['total'])
    worksheet.append_row(row)

In [0]:
#export to google sheets

def export_to_overview(customer, audiences, dates, report):
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1je3b1g6Yg2B-bmwx6CP5akcrsj9DzQcs9PnKHoKoioQ/edit').sheet1
    row = overview_row(customer, audiences, dates, report['total'])
    worksheet.append_row(row)

## Version
Version of the analysis script corresponding to the methodology version in the whitepaper (Major + Minor version represent the whitepaper version, revision represents changes and fixes of the uplift report script).

In [23]:
display(__version__)

'5.4.15-internal'

## Load CSV data from S3

Load mark, spend and event data from S3. 

### IMPORTANT

**The event data is usually quite large (several GB) so this operation might take several minutes or hours to complete, depending on the size and connection.**

### Deduplication for appsflyer
AppsFlyer sends some events twice - we want to remove the duplicates before the analysis

In [0]:

    # AppsFlyer sends some events twice - we want to remove the duplicates before the analysis
     if use_deduplication:
        df = drop_duplicates_in_attributions(df, pd.Timedelta('1 minute'))
    


### Calculate and display uplift report for the data set as a whole

This takes the whole data set and calculates uplift KPIs.

In [15]:
marks_and_spend_df = load_marks_and_spend_data(customer, audiences, dates)


2019-09-03 12:31:07.433842 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_iOS/marks_and_spend/cache-v5.4.15-internal/20190820.parquet
2019-09-03 12:31:07.433842 stored S3 cache file to local drive, loading cache-v5.4.15-internal/3_Lovoo_iOS/marks_and_spend/20190820.parquet
2019-09-03 12:31:08.107843 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_iOS/marks_and_spend/cache-v5.4.15-internal/20190821.parquet
2019-09-03 12:31:08.107843 stored S3 cache file to local drive, loading cache-v5.4.15-internal/3_Lovoo_iOS/marks_and_spend/20190821.parquet
2019-09-03 12:31:08.334577 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_iOS/marks_and_spend/cache-v5.4.15-internal/20190822.parquet
2019-09-03 12:31:08.334577 stored S3 cache file to local drive, loading cache-v5.4.15-internal/3_Lovoo_iOS/marks_and_spend/20190822.parquet
2019-09-03 12:31:08.569663 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_

In [16]:
attributions_df = load_attribution_data(customer, audiences, dates, revenue_event, marks_and_spend_df, use_deduplication)


2019-09-03 12:31:10.448987 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_iOS/attributions/cache-v5.4.15-internal/20190820-Purchase.parquet
2019-09-03 12:31:10.448987 stored S3 cache file to local drive, loading cache-v5.4.15-internal/3_Lovoo_iOS/attributions/20190820-Purchase.parquet
2019-09-03 12:31:10.777529 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_iOS/attributions/cache-v5.4.15-internal/20190821-Purchase.parquet
2019-09-03 12:31:10.777529 stored S3 cache file to local drive, loading cache-v5.4.15-internal/3_Lovoo_iOS/attributions/20190821-Purchase.parquet
2019-09-03 12:31:10.991092 loading from S3 cache s3://remerge-customers/lovoo-2/uplift_data/3_Lovoo_iOS/attributions/cache-v5.4.15-internal/20190822-Purchase.parquet
2019-09-03 12:31:10.991092 stored S3 cache file to local drive, loading cache-v5.4.15-internal/3_Lovoo_iOS/attributions/20190822-Purchase.parquet
2019-09-03 12:31:11.241755 loading from S3 cache s3://remerge-cu

In [18]:
marks_and_spend_df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71232 entries, 0 to 71231
Data columns (total 6 columns):
ts               71232 non-null int32
event_type       71232 non-null object
ab_test_group    71232 non-null bool
user_id          71232 non-null int64
campaign_id      71232 non-null int64
cost_eur         631 non-null float64
dtypes: bool(1), float64(1), int32(1), int64(2), object(1)
memory usage: 6.1 MB


Print some statistics of the loaded data sets.

In [19]:
attributions_df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8680 entries, 0 to 8679
Data columns (total 3 columns):
ts             8680 non-null int32
user_id        8680 non-null int64
revenue_eur    8578 non-null float64
dtypes: float64(1), int32(1), int64(1)
memory usage: 169.6 KB


## Uplift Results

You can configure the ouput by using variables in the 'Configuration' section

In [0]:
report = uplift_report(marks_and_spend_df, attributions_df, groups, per_campaign_results, use_converters_for_significance)

In [0]:
# set formatting options
pd.set_option('display.float_format', '{:.3f}'.format)

In [22]:
display(report)

,total,c_18789,c_18784
ad spend,272.540,100.940,171.600
total revenue,83138.012,73784.635,9393.838
test group size,25487,19471,6069
test conversions,5175,4640,538
test converters,1005,818,189
test revenue,76110.205,67141.119,9009.547
control group size,2820,2109,718
control conversions,331,299,32
control_converters,108,87,21
control revenue,7027.807,6643.516,384.291


### CSV Export - combined reports

In [24]:
start = dates[0]
end = dates[-1]
export_csv(report,'{}_{}-{}.csv'.format(customer, start, end))

Stored results as a local CSV file: lovoo-2_2019-08-20 00:00:00-2019-08-23 00:00:00.csv
The download of the results file should start automatically


### Group by Date and export

In [27]:
#calculate conversions, customer revnenue grouped


def group_conv_date(marks_and_spend_df, attributions_df): 

    merged_df = merge(marked(marks_and_spend_df),attributions_df)
    merged_df['ts_x']  = pd.to_datetime(merged_df['ts_x'],unit='s').dt.date
    merged_df['conversions'] = merged_df['user_id']
    grouped = merged_df.groupby(['ts_x','campaign_id','ab_test_group'])
    grouped_conv_df = grouped.agg({'revenue_eur':'sum','user_id' :'nunique', 'conversions': 'count'}).reset_index()
    grouped_conv_df['customer revenue'] = grouped_conv_df['revenue_eur'] / 10 ** 6
    return pd.DataFrame(grouped_conv_df).drop(['revenue_eur'], axis =1).rename(columns = { 'ts_x': 'ts', 'ab_test_group':'ab_test_group', 'user_id':'converters'})
                       
table2_df = group_conv_date(marks_and_spend_df,attributions_df)

    



#calculating both group size and ad spend from marks_and_spend_df


def group_spend_size (marks_and_spend_df) :
  marks_and_spend_df['ts']  = pd.to_datetime(marks_and_spend_df['ts'],unit='s').dt.date
  grouped = marks_and_spend_df.groupby(['ts','campaign_id','ab_test_group'])
  grouped_spend_size_df = grouped.agg({'cost_eur':'sum','user_id':'nunique'}).reset_index()
  grouped_spend_size_df['ad spend']=grouped_spend_size_df['cost_eur'] / 10 ** 6
  return pd.DataFrame(grouped_spend_size_df).drop(['cost_eur'], axis =1).rename(columns={'user_id': 'group size'})

table1_df = group_spend_size(marks_and_spend_df)

table3_df = pd.merge(table2_df, table1_df, on = ['ts','campaign_id','ab_test_group'])
print(pd.DataFrame(table3_df))

# export to csv

from google.colab import files

date=dates.strftime('%Y-%m-%d')
start=str(date[0])
end=str(date[-1])
name='{}_{}_{}-{}{}'.format('PerDay',customer,start,end,'.csv')
table3_df.to_csv(name, index = False) 
files.download(name)

  

    
  


  
    





            ts  campaign_id  ...  group size  ad spend
0   2019-08-20        18784  ...         437     0.000
1   2019-08-20        18784  ...        3727    43.800
2   2019-08-20        18789  ...        1321     0.000
3   2019-08-20        18789  ...       12271    24.700
4   2019-08-21        18784  ...         449     0.000
5   2019-08-21        18784  ...        3768    41.800
6   2019-08-21        18789  ...        1278     0.000
7   2019-08-21        18789  ...       11955    26.360
8   2019-08-22        18784  ...         460     0.000
9   2019-08-22        18784  ...        3754    44.200
10  2019-08-22        18789  ...        1283     0.000
11  2019-08-22        18789  ...       11804    23.260
12  2019-08-23        18784  ...         457     0.000
13  2019-08-23        18784  ...        3747    41.800
14  2019-08-23        18789  ...        1348     0.000
15  2019-08-23        18789  ...       12542    26.620

[16 rows x 8 columns]


## Export to overview sheet

In [0]:
def overview_row(customer, audiences, dates, total):
    return list([
        customer,
        ",".join(audiences),
        dates[0].strftime('%Y-%m-%d'),
        dates[-1].strftime('%Y-%m-%d'),
        __version__,
    ]) + list(total.values)

def export_to_overview(customer, audiences, dates, report):
    !pip install --upgrade --quiet gspread
    from google.colab import auth
    auth.authenticate_user()

    import gspread
    from oauth2client.client import GoogleCredentials
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    worksheet = gc.open('Uplift Report Overview').sheet1
    row = overview_row(customer, audiences, dates, report['total'])
    worksheet.append_row(row)
   
export_to_overview(customer, audiences, dates, report)